## Importando bibliotecas necessárias e realizando Web Scraping

### Utilizando **Requests** para fazer requisições HTTP e baixar páginas web.
### Utilizando **beautifulsoup4** para extrair e manipular dados de HTML/XML de forma fácil.

In [2]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"
response = requests.get(url)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}


### Função para  download de arquivo PDF

In [3]:
def download_pdf(pdf_url, path="downloads"):
    os.makedirs(path, exist_ok=True)
    filename = os.path.join(path, pdf_url.split("/")[-1])

    response = requests.get(pdf_url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {pdf_url}")

### Verificando os links, para baixar os arquivos desejados

In [10]:
soup = BeautifulSoup(response.text, "html.parser")
pdf_links = [a["href"] for a in soup.find_all("a", href=True) if ".pdf" in a["href"]]
for link in pdf_links:
    print(f"Link found: {link}")

Link found: https://www.gov.br/ans/pt-br/arquivos/composicao/organograma_ans_geral_setembro_2024-1.pdf
Link found: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Link found: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
Link found: https://www.gov.br/ans/pt-br/arquivos/assuntos/consumidor/o-que-seu-plano-deve-cobrir/nota13_geas_ggras_dipro_17012013.pdf
Link found: https://www.gov.br/ans/pt-br/arquivos/composicao/organograma_ans_geral_setembro_2024-1.pdf


### Sabemos que o primeiro link da lista não é o que queremos, então ignoramos ele e baixamos apenas os próximos dois

In [8]:
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    pdf_links = [a["href"] for a in soup.find_all("a", href=True) if ".pdf" in a["href"]]

    pdf_links = pdf_links[1:]
    
    if len(pdf_links) > 1:
        for link in pdf_links[:2]:
            if link.startswith("/"):
                link = url + link
            download_pdf(link)
    else:
        print("No PDFs found")
else:
    print("Failed to fetch page")

Downloaded downloads/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Downloaded downloads/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf


### Compactando os Arquivos em ZIP

In [9]:
import zipfile
import os

def compress_zip(path="downloads", name_compress_zip="anexos.zip"):
    path_zip = os.path.join(path, name_compress_zip)

    with zipfile.ZipFile(path_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(path):
            for file in files:
                path_zip = os.path.join(root, file)
                zipf.write(path_zip, os.path.relpath(path_zip, path))
    print(f"Compressed {path_zip}")

compress_zip()
    

Compressed downloads/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
